In [5]:
!pip3 install -U torch

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: torch in /home/emr-notebook/.local/lib/python3.7/site-packages (1.12.0)


In [6]:
# restart kernel to pick up the pip installs above
import IPython

IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

{'status': 'ok', 'restart': True}

In [26]:
!aws s3 cp --recursive s3://dsoaws/ray_output/ ~/

download: s3://dsoaws/ray_output/TorchTrainer_2022-08-06_01-49-34/TorchTrainer_05a2a_00000_0_2022-08-06_01-49-34/checkpoint_000000/.is_checkpoint to home/emr-notebook/TorchTrainer_2022-08-06_01-49-34/TorchTrainer_05a2a_00000_0_2022-08-06_01-49-34/checkpoint_000000/.is_checkpoint
download: s3://dsoaws/ray_output/TorchTrainer_2022-08-06_01-49-34/TorchTrainer_05a2a_00000_0_2022-08-06_01-49-34/events.out.tfevents.1659750574.ip-172-31-20-207 to home/emr-notebook/TorchTrainer_2022-08-06_01-49-34/TorchTrainer_05a2a_00000_0_2022-08-06_01-49-34/events.out.tfevents.1659750574.ip-172-31-20-207
download: s3://dsoaws/ray_output/TorchTrainer_2022-08-06_01-49-34/TorchTrainer_05a2a_00000_0_2022-08-06_01-49-34/checkpoint_000000/.tune_metadata to home/emr-notebook/TorchTrainer_2022-08-06_01-49-34/TorchTrainer_05a2a_00000_0_2022-08-06_01-49-34/checkpoint_000000/.tune_metadata
download: s3://dsoaws/ray_output/TorchTrainer_2022-08-06_01-49-34/basic-variant-state-2022-08-06_01-49-34.json to home/emr-noteboo

In [27]:
model_path = '/home/emr-notebook/TorchTrainer_2022-08-06_01-49-34/TorchTrainer_05a2a_00000_0_2022-08-06_01-49-34/checkpoint_000000'

In [28]:
import torch
from torch import nn

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [29]:
from torchvision import datasets
from torchvision.transforms import ToTensor

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="~/data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [30]:
import torch
from torch import nn

from ray.train.torch import TorchCheckpoint

def predict_from_model(model):
    classes = [
        "T-shirt/top",
        "Trouser",
        "Pullover",
        "Dress",
        "Coat",
        "Sandal",
        "Shirt",
        "Sneaker",
        "Bag",
        "Ankle boot",
    ]

    model.eval()
    x, y = test_data[0][0], test_data[0][1]
    with torch.no_grad():
        pred = model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')
        
model_loaded = TorchCheckpoint(local_path=model_path).get_model(NeuralNetwork())

predict_from_model(model_loaded)

Predicted: "Ankle boot", Actual: "Ankle boot"
